# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file=os.path.join('..','WeatherPy','output_data','Clean_City_data.csv')
df=pd.read_csv(file)
del df['Unnamed: 0']
df.head()


,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed,Country,Date
0,chicama,-7.84,-79.15,64.02,80.0,26.0,8.14,PE,1.592015e+09
1,tumut,-35.30,148.22,59.40,71.0,100.0,4.94,AU,1.592015e+09
2,mahebourg,-20.41,57.70,71.60,88.0,75.0,10.29,MU,1.592014e+09
3,lagoa,39.05,-27.98,64.33,87.0,100.0,16.20,PT,1.592014e+09
4,cape town,-33.93,18.42,50.65,87.0,40.0,13.87,ZA,1.592014e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = df[["Lat", "Lng"]]
humidity=df["Humidity (%)"]

# Plot Heatmap
fig = gmaps.figure(map_type='TERRAIN',center=(0, 0), zoom_level=2,layout={
        'width': '1000px',
        'height': '700px',
        'padding': '3px',
        'border': '1px solid black'
})

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', padding='3px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
ideal_df=df.loc[(df["Max Temp (F)"]>=70) & (df["Max Temp (F)"]<=80) & (df["Wind Speed"]<=10) & (df["Cloudiness (%)"]==0),:]
ideal_df=ideal_df.reset_index(drop=True)
ideal_df.head(20)

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed,Country,Date
0,pimenta bueno,-11.67,-61.19,72.25,91.0,0.0,2.15,BR,1.592015e+09
1,surab,28.49,66.26,72.39,17.0,0.0,3.98,PK,1.592015e+09
2,chardara,41.25,67.97,74.77,42.0,0.0,5.35,KZ,1.592015e+09
3,hami,42.80,93.45,77.05,17.0,0.0,3.09,CN,1.592014e+09
4,maceio,-9.67,-35.74,71.60,88.0,0.0,8.48,BR,1.592014e+09
5,tahta,26.77,31.50,78.87,21.0,0.0,8.05,EG,1.592015e+09
6,kedrovyy,56.17,91.82,71.60,60.0,0.0,6.71,RU,1.592014e+09
7,caravelas,-17.71,-39.25,70.90,92.0,0.0,4.72,BR,1.592014e+09
8,diamantino,-14.41,-56.45,73.99,62.0,0.0,4.00,BR,1.592015e+09
9,buctzotz,21.20,-88.78,75.70,91.0,0.0,3.58,MX,1.592015e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotels=[]

# Loop through dataframe to get nearest hotel
for city in range(len(ideal_df["City"])):
    lat = ideal_df.loc[city]["Lat"]
    lng = ideal_df.loc[city]["Lng"]
    city_coords = f"{lat},{lng}"
    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
              }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    hotel_response = requests.get(base_url, params=params).json()
 
    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
ideal_df["Nearest Hotel"] = hotels
ideal_df.head()



,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed,Country,Date,Nearest Hotel
0,pimenta bueno,-11.67,-61.19,72.25,91.0,0.0,2.15,BR,1.592015e+09,Cristalina Palace Hotel
1,surab,28.49,66.26,72.39,17.0,0.0,3.98,PK,1.592015e+09,Bismillah Hotel
2,chardara,41.25,67.97,74.77,42.0,0.0,5.35,KZ,1.592015e+09,Жанаргул
3,hami,42.80,93.45,77.05,17.0,0.0,3.09,CN,1.592014e+09,Laoyutang Resort
4,maceio,-9.67,-35.74,71.60,88.0,0.0,8.48,BR,1.592014e+09,Mercure Maceio Pajucara Hotel


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = ideal_df[["Lat", "Lng"]]


In [7]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Hotel: {hotel}" for hotel in hotels])
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='700px', padding='3px', width='1000px'))